In [1]:
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator,Sampler

from squlearn.util.optree.optree_derivative import derivative
from squlearn.util.optree.optree_evaluate import build_operator_list,evaluate_estimator,evaluate_sampler
from squlearn.util.optree import *

from squlearn.expectation_operator import SummedPaulis
from squlearn.feature_map import ChebRx

In [2]:
op = SummedPaulis(2)
pop=ParameterVector('pop', op.num_parameters)
e = simplify_copy(op.get_pauli_new(pop).power(2))
print(e)

SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[ParameterExpression(1.0*pop[0]**2 + 1.0*pop[1]**2 + 1.0*pop[2]**2),
 ParameterExpression(2.0*pop[0]*pop[1]),
 ParameterExpression(2.0*pop[0]*pop[2]),
 ParameterExpression(2.0*pop[1]*pop[2])])


In [3]:
op_grad = derivative(e,pop)
print(op_grad)

op_grad = OpTreeNodeSum([op_grad,op_grad])

[1.0*SparsePauliOp(['II', 'IZ', 'ZI'],
              coeffs=[ParameterExpression(2.0*pop[0]), ParameterExpression(2.0*pop[1]),
 ParameterExpression(2.0*pop[2])]), 1.0*SparsePauliOp(['II', 'IZ', 'ZZ'],
              coeffs=[ParameterExpression(2.0*pop[1]), ParameterExpression(2.0*pop[0]),
 ParameterExpression(2.0*pop[2])]), 1.0*SparsePauliOp(['II', 'ZI', 'ZZ'],
              coeffs=[ParameterExpression(2.0*pop[2]), ParameterExpression(2.0*pop[0]),
 ParameterExpression(2.0*pop[1])])]


In [4]:
fm = ChebRx(2,1)
x = ParameterVector("x",fm.num_features)
p = ParameterVector("p",fm.num_parameters)
fm_circ = fm.get_circuit(x,p)

fm_grad = derivative(fm_circ,p)
print(fm_grad)

[1.0*(0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) + π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
 + -0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) - π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
), 1.0*(0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────┴──┐├──────────┤┌─┴─┐
q_1: ┤ Rx(p[1]*acos(x[0]) + π/2) ├┤ Rx(p[3]) ├┤ X ├
     └───────────────────────────┘└──────────┘└───┘
 + -0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────

In [5]:
#dictionary1={pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0, x[0]:-0.5,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}

#estim = Estimator()

#evaluate_estimator(fm_grad,op_grad,dictionary1,estim)

In [6]:
dictionary_c1 = {x[0]:-0.5,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}
dictionary_c2 = {x[0]: 0.1,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}

dictionary_op = {pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0}

estim = Estimator()

evaluate_estimator(fm_grad,op_grad,[dictionary_c1,dictionary_c2],dictionary_op,estim)

run time 0.07799911499023438
estimator_result [-1.16993962  2.82991043  2.85386524 -1.16993962  2.82991043  2.85386524
  3.16993962  4.0851134   4.08240263  3.16993962  4.0851134   4.08240263
  1.84855816  1.86847731  1.53283493  1.84855816  1.86847731  1.53283493
  2.48704612  2.68859221  2.46716507  2.48704612  2.68859221  2.46716507
 -1.16993962  2.82991043  2.85386524 -1.16993962  2.82991043  2.85386524
  3.16993962  4.0851134   4.08240263  3.16993962  4.0851134   4.08240263
  1.84855816  1.86847731  1.53283493  1.84855816  1.86847731  1.53283493
  2.48704612  2.68859221  2.46716507  2.48704612  2.68859221  2.46716507
 -1.26012278  2.74974076  2.800815   -1.26012278  2.74974076  2.800815
  3.26012278  4.07450186  4.06736696  3.26012278  4.07450186  4.06736696
  1.68448602  1.66282968  1.34061395  1.68448602  1.66282968  1.34061395
  2.56301661  2.83550541  2.65938605  2.56301661  2.83550541  2.65938605
 -1.26012278  2.74974076  2.800815   -1.26012278  2.74974076  2.800815
  3.26012

array([[[-9.08942181, -2.62889096, -2.57304268],
        [-1.33724604, -1.71764462, -1.95685649],
        [-4.33987923, -1.25520297, -1.22853739],
        [-0.63848795, -0.8201149 , -0.93433015]],

       [[-6.64760379, -1.94823196, -1.86262793],
        [-1.29199249, -1.72457083, -1.93942436],
        [-4.52024557, -1.3247611 , -1.26655196],
        [-0.8785306 , -1.17267573, -1.3187721 ]]])

In [7]:
dictionary_c1 = {x[0]:-0.5,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}
dictionary_c2 = {x[0]: 0.1,p[0]:10.0,p[1]:20.0,p[2]:30.0,p[3]:40.0}

dictionary_op1 = {pop[0]: 0.5,pop[1]:0.75, pop[2]:1.0}
dictionary_op2 = {pop[0]: 0.25,pop[1]:0.25, pop[2]:0.5}

estim = Estimator()

evaluate_estimator(fm_grad,op_grad,[dictionary_c1,dictionary_c2],[dictionary_op1,dictionary_op2],estim,dictionaries_combined=True)

run time 0.06499600410461426
estimator_result [-1.16993962  2.82991043  2.85386524 -1.16993962  2.82991043  2.85386524
  3.16993962  4.0851134   4.08240263  3.16993962  4.0851134   4.08240263
  1.84855816  1.86847731  1.53283493  1.84855816  1.86847731  1.53283493
  2.48704612  2.68859221  2.46716507  2.48704612  2.68859221  2.46716507
 -1.16993962  2.82991043  2.85386524 -1.16993962  2.82991043  2.85386524
  3.16993962  4.0851134   4.08240263  3.16993962  4.0851134   4.08240263
  1.84855816  1.86847731  1.53283493  1.84855816  1.86847731  1.53283493
  2.48704612  2.68859221  2.46716507  2.48704612  2.68859221  2.46716507
 -0.46446626  1.12487038  1.16139234 -0.46446626  1.12487038  1.16139234
  1.46446626  1.78725093  1.79466832  1.46446626  1.78725093  1.79466832
  0.65495112  0.58141484  0.74359921  0.65495112  0.58141484  0.74359921
  1.09421642  1.1677527   1.25640079  1.09421642  1.1677527   1.25640079
 -0.46446626  1.12487038  1.16139234 -0.46446626  1.12487038  1.16139234
  1.4

array([[[-9.08942181, -2.62889096, -2.57304268],
        [-1.33724604, -1.71764462, -1.95685649],
        [-4.33987923, -1.25520297, -1.22853739],
        [-0.63848795, -0.8201149 , -0.93433015]],

       [[-2.83674391, -0.97411598, -0.93131396],
        [-0.64599625, -0.86228541, -0.75414083],
        [-1.92893251, -0.66238055, -0.63327598],
        [-0.4392653 , -0.58633786, -0.51280158]]])

In [8]:
from squlearn.util import Executor
from squlearn.qnn.qnn import QNN

executor = Executor(Estimator())
qnn = QNN(fm,op,executor)

res = qnn.evaluate_from_string("dfdopdp",x=[-0.5,0.1],param=[10,20,30,40],param_op=[0.5,0.75,1.0])



print(res[0].transpose())
print(res[1].transpose())

[[ 0.00000000e+00 -1.31444548e+00 -1.28652134e+00]
 [ 0.00000000e+00  2.22044605e-16 -3.34311511e-01]
 [ 0.00000000e+00 -6.27601486e-01 -6.14268693e-01]
 [ 5.55111512e-17  5.55111512e-17 -1.59621989e-01]]
[[ 1.11022302e-16 -9.74115981e-01 -9.31313963e-01]
 [ 0.00000000e+00  1.11022302e-16 -3.22998123e-01]
 [ 0.00000000e+00 -6.62380548e-01 -6.33275981e-01]
 [ 1.11022302e-16  1.66533454e-16 -2.19632649e-01]]


In [9]:
print(fm_grad)

[1.0*(0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) + π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
 + -0.5*acos(x[0])*
     ┌───────────────────────────┐┌──────────┐     
q_0: ┤ Rx(p[0]*acos(x[0]) - π/2) ├┤ Rx(p[2]) ├──■──
     └──┬─────────────────────┬──┘├──────────┤┌─┴─┐
q_1: ───┤ Rx(p[1]*acos(x[0])) ├───┤ Rx(p[3]) ├┤ X ├
        └─────────────────────┘   └──────────┘└───┘
), 1.0*(0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────┴──┐├──────────┤┌─┴─┐
q_1: ┤ Rx(p[1]*acos(x[0]) + π/2) ├┤ Rx(p[3]) ├┤ X ├
     └───────────────────────────┘└──────────┘└───┘
 + -0.5*acos(x[0])*
        ┌─────────────────────┐   ┌──────────┐     
q_0: ───┤ Rx(p[0]*acos(x[0])) ├───┤ Rx(p[2]) ├──■──
     ┌──┴─────────────────────

In [10]:
sampler = Sampler()

evaluate_sampler(fm_grad,op_grad,[dictionary_c1,dictionary_c2],[dictionary_op1,dictionary_op2],sampler,dictionaries_combined=False)

IndexError: list index out of range